# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re

In [27]:
# Download NLTK resources (run only once)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/aryanjain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aryanjain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:


# Step 1: Read the raw file content
file_path = "data/reviews.txt"

# Open the file and process each line
processed_lines = []
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        # Remove the line number and period at the start of the line
        if line.strip():  # Skip empty lines
            processed_line = line.split(". ", 1)[-1].strip()
            # Remove the surrounding quotes
            processed_line = processed_line.strip('"')
            processed_lines.append(processed_line)

# Step 2: Create a DataFrame from the processed lines
reviews = pd.DataFrame(processed_lines, columns=["ReviewText"])

# Step 3: Add a ReviewNumber column
reviews.insert(0, "ReviewNumber", range(1, len(reviews) + 1))




# Display the first few rows to verify
print(reviews.head())

   ReviewNumber                                         ReviewText
0             1  Absolutely breathtaking views and fantastic sk...
1             2  The lodge was cozy, and the staff was incredib...
2             3  Mountain biking trails were well-maintained, a...
3             4  Loved the guided nature walk. Learned so much ...
4             5  The zip-lining course was thrilling, and the i...


In [11]:

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    return " ".join(tokens)



In [12]:
# Apply preprocessing to the ReviewText column
reviews["CleanedText"] = reviews["ReviewText"].apply(preprocess_text)

# Step 3: Create a document-term matrix
vectorizer = CountVectorizer(max_features=5000)  # Limit to top 5000 words
X = vectorizer.fit_transform(reviews["CleanedText"]).toarray()

# Step 4: Label the data (manual labeling for training)
# Example labels: Assume we manually label the first 100 reviews
train_indices = range(100)  # First 100 reviews for training
train_data = X[train_indices]
train_labels = ["positive"] * 40 + ["neutral"] * 30 + ["negative"] * 30  # Example labels



In [14]:
# Step 5: Train the model (Naive Bayes)
model = MultinomialNB()
model.fit(train_data, train_labels)



MultinomialNB()

In [15]:
# Step 6: Evaluate the model
# Use the remaining data for testing
test_data = X[100:]  # Reviews after the first 100
test_labels = ["positive"] * 40 + ["neutral"] * 30 + ["negative"] * 30  # Example labels



In [17]:
print("Test Labels:")
print(test_labels[:10])  # First 10 labels
print("Predictions:")
print(predictions[:10])  # First 10 predictions

print("\nShapes:")
print("Test Labels Shape:", len(test_labels))
print("Predictions Shape:", len(predictions))

Test Labels:
['positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive']
Predictions:
['negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'positive' 'neutral' 'negative' 'negative']

Shapes:
Test Labels Shape: 100
Predictions Shape: 425


In [18]:
from sklearn.preprocessing import LabelEncoder

# Initialize a label encoder
label_encoder = LabelEncoder()

# Fit the encoder on the training labels (if not already done)
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform test_labels and predictions to match the encoding
test_labels_encoded = label_encoder.transform(test_labels)
predictions_encoded = label_encoder.transform(predictions)

In [20]:
print("Length of test_labels:", len(test_labels))
print("Length of predictions:", len(predictions))

Length of test_labels: 100
Length of predictions: 425


In [21]:
# Ensure test_labels and predictions are aligned
if len(test_labels) != len(predictions):
    # Assuming predictions is for the entire dataset, extract the test set predictions
    predictions = predictions[-len(test_labels):]  # Adjust slicing as needed

In [22]:
print("Aligned Length of test_labels:", len(test_labels))
print("Aligned Length of predictions:", len(predictions))

Aligned Length of test_labels: 100
Aligned Length of predictions: 100


In [23]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit the encoder on the training labels (if not already done)
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Transform test_labels and predictions
test_labels_encoded = label_encoder.transform(test_labels)
predictions_encoded = label_encoder.transform(predictions)

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

print("Confusion Matrix:")
print(confusion_matrix(test_labels_encoded, predictions_encoded))

print("\nClassification Report:")
print(classification_report(test_labels_encoded, predictions_encoded))

Confusion Matrix:
[[11  5 14]
 [13  6 11]
 [19  5 16]]

Classification Report:
              precision    recall  f1-score   support

           0       0.26      0.37      0.30        30
           1       0.38      0.20      0.26        30
           2       0.39      0.40      0.40        40

    accuracy                           0.33       100
   macro avg       0.34      0.32      0.32       100
weighted avg       0.35      0.33      0.33       100



In [ ]:
# Step 7: Predict sentiment for all reviews
all_predictions = model.predict(X)
reviews["Sentiment"] = all_predictions

# View the results
print(reviews[["ReviewText", "Sentiment"]].head())

In [31]:
%pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 14.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
